In [1]:
import pandas as pd
import numpy as np
import demoji
import nltk
import os

In [2]:
df = pd.read_json('dataset/pavbhaji.json')

In [ ]:
df.head()

In [ ]:
183+269

In [ ]:
df.columns

In [ ]:
df['tags'].head()

In [3]:
df['thumbnail_src'] = df['thumbnail_src'].apply(lambda x: x.split('/')[-1])

In [ ]:
df['thumbnail_src'][1]

In [ ]:
df['urls']

In [4]:
df['urls'] = df['urls'].apply(lambda x: x[0].split('/')[-1]if len(x)>0 else np.nan)

In [ ]:
x = df['edge_media_to_caption'][1]['edges'][0]['node']['text']

In [ ]:
demoji.download_codes()

In [5]:
def extract_text(df):
    df['edge_media_to_caption'] = df['edge_media_to_caption'].apply(lambda x: demoji.
                                                                    replace(x['edges'][0]['node']['text']) 
                                                                    if len(x['edges']) > 0 else np.nan )
    return df
                                                                    

In [6]:
df = extract_text(df)

In [7]:
pavbhaji_images = os.listdir('dataset/images/0')
nopavbhaji_images = os.listdir('dataset/images/1')

In [8]:
images = pavbhaji_images+nopavbhaji_images

In [9]:
data_df = df[df['thumbnail_src'].isin(images)]

In [10]:
data_df['pavbhaji']=np.nan

/home/sanket/anaconda3/envs/udacity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
data_df['pavbhaji'] = data_df['thumbnail_src'].apply(lambda x: 1 if x in pavbhaji_images else 0)

/home/sanket/anaconda3/envs/udacity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data_df.isnull().sum()

In [ ]:
data_df[data_df['thumbnail_src']==pavbhaji_images[0]]

In [15]:
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import multioutput
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
#ML models
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pickle

In [14]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/sanket/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sanket/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sanket/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data_df.head()

In [12]:
data_df['tags'] = data_df['tags'].apply(lambda x: list(x))

/home/sanket/anaconda3/envs/udacity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
data_df['tags'] = data_df['tags'].apply(lambda x: ' '.join(x))

/home/sanket/anaconda3/envs/udacity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
data_df.shape

In [ ]:
x=['Pavbhaji','NoPavphaji']
pavbhaji_y = data_df['pavbhaji'].sum()
y=[pavbhaji_y,452-pavbhaji_y]
trace1 = go.Bar(x=x,y=y,orientation='v',marker=dict(color='#ffdc51'),name='Data Distribution')
layout = go.Layout(title="Data distribution across different label categories"
                   ,legend=dict(x=0.1, y=1.1, orientation="h")
                   ,xaxis=dict(title="Label Name"),
                   yaxis=dict(title="Number of data points for each label"))
fig = go.Figure(data=[trace1],layout=layout)
iplot(fig)

In [ ]:
data_df.columns

# Machine learning models 

In [ ]:
X = data_df['tags']
y = data_df.iloc[:,-1]

In [ ]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [ ]:
X.shape,y.shape

In [ ]:
def tokenize(text):
    '''
    Input
    text: take the text as input
    Output
    words_lemmed: tokenized and lemmatized text with stop words removed 
    '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    stop_words = stopwords.words("english")
    
    #tokenize
    words = word_tokenize (text)
    words_lemmed = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stop_words]
    return words_lemmed

In [ ]:
X_train.shape,y_train.shape

In [ ]:
v = CountVectorizer(tokenizer=tokenize)
x_v = v.fit_transform(X)

In [ ]:
tf = TfidfTransformer()
x_tf = tf.fit_transform(x_v)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_tf, y, random_state = 22)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)
np.mean(predicted == np.array(y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=22)

In [ ]:
clf_rf.fit(X_train,y_train)

In [ ]:
y_predicted_rf = clf_rf.predict(X_test)
np.mean(y_predicted_rf == np.array(y_test))

In [ ]:
y_predicted_rf,np.array(y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=22)
clf_lr.fit(X_train,y_train)

In [ ]:
y_predicted_lr = clf_lr.predict(X_test)
np.mean(y_predicted_rf == np.array(y_test))

In [ ]:
y_predicted_lr,np.array(y_test)

In [ ]:
from keras.models import Sequential
from keras import layers
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
x_array = X_train.toarray()
y_array = np.array(y_train)
x_test_array = X_test.toarray()
y_test_array = np.array(y_test)

In [ ]:
history = model.fit(x_array, y_array,epochs=100,verbose=False,validation_data=(x_test_array, y_test_array),batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(x_array, y_array, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test_array, y_test_array, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
data_df.columns

In [ ]:
data_df.head()

In [16]:
def extract_text(df):
    df['edge_media_to_caption'] = df['edge_media_to_caption'].apply(lambda x:x.replace("\n"," ") if len(x)>0 else x)
    #df['edge_media_to_caption'] = df['edge_media_to_caption'].apply(lambda x:x.replace("@"," ") if len(x) >0 else x)        
    return df

In [17]:
# data_df.head()
a = data_df.copy()
a = extract_text(a)
a = a.reset_index(drop=True)

In [19]:
data_df = a.copy()

In [20]:
import re,string
def strip_all_entities(text):
    entity_prefixes = ['@','@_','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [ ]:
data_df['caption']

In [ ]:
strip_all_entities(a['edge_media_to_caption'][0])

In [ ]:
data_df['edge_media_to_caption']

In [21]:
data_df['caption'] = data_df['edge_media_to_caption'].apply(lambda x: strip_all_entities(x))

In [22]:
data_df = data_df.reset_index()

In [23]:
data_df['text'] = data_df['tags']+ " " + data_df['caption']

In [ ]:
data_df['tags'][0], data_df['caption'][0]

In [24]:
X = data_df['text']
y = data_df.iloc[:,-3]

In [25]:
y

0      1
1      0
2      1
3      1
4      1
      ..
447    0
448    0
449    0
450    1
451    1
Name: pavbhaji, Length: 452, dtype: int64

In [26]:
vect = CountVectorizer(tokenizer=tokenize)
X = vect.fit_transform(X)
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)

NameError: name 'tokenize' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

In [ ]:
pipeline = Pipeline([('scale',StandardScaler(with_mean=False)),
                     ('clf', LinearSVC())])

In [ ]:
search_space = [{'clf':[LinearSVC()]},
                
                {'clf': [LogisticRegression(solver='sag',random_state=22)]},
                
                {'clf': [MultinomialNB()]},
                {'clf':[RandomForestClassifier(n_estimators=200, max_depth=3, random_state=22)]}]

In [ ]:
cv = GridSearchCV(pipeline, search_space)

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
pred = cv.predict(X_test)
np.mean(pred == np.array(y_test))

In [ ]:
x_train_array = X_train.toarray()
y_train_array = np.array(y_train)
x_test_array = X_test.toarray()
y_test_array = np.array(y_test)

In [27]:
from keras.models import Sequential
from keras import layers


In [ ]:
input_dim = X_train.shape[1]  # Number of features
model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
scaler = StandardScaler()
# transform data
x_train_array = scaler.fit_transform(x_train_array)
x_test_array = scaler.fit_transform(x_test_array)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train_array, y_train_array,epochs=100,verbose=False,validation_data=(x_test_array, y_test_array),batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(x_train_array, y_train_array, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(x_test_array, y_test_array, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


In [ ]:
plot_history(history)

## Through LSTM

In [33]:
from keras.preprocessing import sequence

In [ ]:
vocabulary_size = len(vect.vocabulary_)

In [ ]:
max_words = 50
X_train_rnn = sequence.pad_sequences(x_train_array, maxlen=max_words)
X_test_rnn = sequence.pad_sequences(x_test_array, maxlen=max_words)

In [31]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,SpatialDropout1D


In [ ]:
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [29]:
from keras.preprocessing.text import Tokenizer

In [34]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data_df['text'].values)
X_rnn = tokenizer.texts_to_sequences(data_df['text'].values)
X_rnn = sequence.pad_sequences(X_rnn)

In [35]:
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_rnn, y, random_state = 22)

In [36]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train_rnn.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 295, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 295, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 4
model.fit(X_train_rnn, y_train_rnn, epochs = 7,validation_data=(X_test_rnn, y_test_rnn), batch_size=batch_size,verbose=False)